# TP NLP: Fouille du web et extraction d'informations

Auteurs:

- GANIYU Isaac

- JUMBONG Junior

### Installation des packages et données nécessaires

In [ ]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json
!pip install feedparser
!pip install newspaper3k
!python -m spacy download fr_core_news_md

--2024-01-29 22:14:32--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5887584 (5.6M) [application/json]
Saving to: ‘francetvinfo.json’

francetvinfo.json   100%[===================>]   5.61M  2.91MB/s    in 1.9s    

2024-01-29 22:14:34 (2.91 MB/s) - ‘francetvinfo.json’ saved [5887584/5887584]

--2024-01-29 22:14:34--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179 [application/json]
Saving to: ‘tvinfo-sources.json’

tvinfo-sources.json 100%[===================>]     179  --.-KB/s    in 0s      

2024-01-29 22:14:35 (5

In [ ]:
import spacy
nlp = spacy.load('fr_core_news_md')
vocab = nlp.vocab

### Acquisition des données

#### Question 1

In [ ]:
import json

with open('francetvinfo.json') as f :
  data_francetv = json.load(f)

print(len(data_francetv))

2097


Le fichier json est structuré comme un dictionnaire de dictionnaire. La clé pour le dictionnaire pincipal est le lien de l'article correspondant et sa valeur est un dictionnaire de clés : le titre de l'article, sa date de parution, l'auteur, la catégorie, le contenu de l'article et le lien de l'image avec à chaque clé la valeur correspondante.

In [ ]:
# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data_lemonde = fp.parse(url)
print(data_lemonde)
print(f'les clés du dictionnaire sont : {data_lemonde.keys()}')
print(f'les clés des dictionnaires de la clé "entries" sont : {data_lemonde["entries"][0].keys()}')
print(f'les clés correspondant à la clé "feed" sont : {data_lemonde["feed"].keys()}')
print(f'les clés correspondant à la clé "headers" sont : {data_lemonde["headers"].keys()}')
print(json.dumps(data_lemonde, indent=2)) # Permet d'afficher de manière lisible la structure du fichier (clés, sous clés, etc.)


{'bozo': False, 'entries': [{'title': 'En direct, colère des agriculteurs\xa0: de «\xa0nouvelles mesures seront prises dès demain\xa0», assure le gouvernement, alors que les blocages se poursuivent en Ile-de-France', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.lemonde.fr/rss/une.xml', 'value': 'En direct, colère des agriculteurs\xa0: de «\xa0nouvelles mesures seront prises dès demain\xa0», assure le gouvernement, alors que les blocages se poursuivent en Ile-de-France'}, 'published': 'Mon, 29 Jan 2024 19:22:08 +0100', 'published_parsed': time.struct_time(tm_year=2024, tm_mon=1, tm_mday=29, tm_hour=18, tm_min=22, tm_sec=8, tm_wday=0, tm_yday=29, tm_isdst=0), 'summary': 'Des centaines d’agriculteurs barrent depuis lundi de grands axes autoroutiers desservant Paris, avec des «\xa0points de blocage\xa0» qui persistent notamment sur l’A1, l’A4, l’A5, l’A6 et l’A13. Les présidents de la FNSEA et des Jeunes Agriculteurs ont été reçus lundi soir par le premier 

Les données retournées par la commande fp.parse correspondent au dictionnaire data_lemonde. ce dernier contient 11 clés ['bozo', 'entries', 'feed', 'headers', 'updated', 'updated_parsed', 'href', 'status', 'encoding', 'version', 'namespaces']. Parmi ces clés, la clé "entries" correspond à une liste de dictionnaires, chacun contenant les clés ['title', 'title_detail', 'published', 'published_parsed', 'summary', 'summary_detail', 'id', 'guidislink', 'link', 'links', 'media_content', 'content', 'media_credit', 'credit']. De même les clés feed et headers sont des dictionnaires de clés respectives : ['title', 'title_detail', 'subtitle', 'subtitle_detail', 'rights', 'rights_detail', 'links', 'link', 'published', 'published_parsed', 'language'] et ['connection', 'content-length', 'content-security-policy', 'cache-control', 'last-modified', 'expires', 'content-type', 'x-xss-protection', 'x-frame-options', 'x-content-type-options', 'via', 'content-encoding', 'accept-ranges', 'date', 'age', 'x-served-by', 'x-cache', 'x-cache-hits', 'x-timer', 'vary', 'strict-transport-security']

In [ ]:
# (c) visualiser les elements du fichier RSS
print(data_lemonde.feed.title)
print(data_lemonde.feed.published)

# (d) iterer sur les entrees du flux RSS
for item in data_lemonde.entries:
  print(item.title, item.published, item.link)

Le Monde.fr - Actualités et Infos en France et dans le monde
Mon, 29 Jan 2024 22:12:26 +0000
En direct, colère des agriculteurs : de « nouvelles mesures seront prises dès demain », assure le gouvernement, alors que les blocages se poursuivent en Ile-de-France Mon, 29 Jan 2024 19:22:08 +0100 https://www.lemonde.fr/economie/live/2024/01/29/en-direct-colere-des-agriculteurs-de-nouvelles-mesures-seront-prises-des-demain-assure-le-gouvernement-les-syndicats-recus-ce-soir-a-matignon_6213513_3234.html
Enquête vidéo : comment Israël détruit les cimetières de Gaza Mon, 29 Jan 2024 19:00:19 +0100 https://www.lemonde.fr/videos/video/2024/01/29/enquete-video-comment-israel-detruit-les-cimetieres-de-gaza_6213717_1669088.html
Planification écologique : Gabriel Attal attendu lors de son discours de politique générale Mon, 29 Jan 2024 18:41:48 +0100 https://www.lemonde.fr/planete/article/2024/01/29/planification-ecologique-gabriel-attal-attendu-lors-de-son-discours-de-politique-generale_6213711_3244.h

In [ ]:
# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


import feedparser as fp

# Récupérer les liens de chaque flux RSS listés dans le fichier
with open('tvinfo-sources.json') as f :
  urls = json.load(f)

# Pour chaque flux RSS, afficher le fichier json et sa structure
for url in urls.values() :
  print(100*"_"+f"\n{url}")
  data = fp.parse(url)
  print(data)
  print(json.dumps(data, indent=2))




____________________________________________________________________________________________________
https://www.francetvinfo.fr/france.rss
{'bozo': False, 'entries': [{'title': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration"}, 'summary': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants.", 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants."}, 'lastpublicati

In [ ]:
# Scanner les articles et réperer ceux qui ne sont pas déjà présents dans la base de données

def isinjson(lien, data):
  """
  Cette fonction retourne un booléen qui indique si le lien donné en entrée se trouve en clé du fichier json donné en entrée.
  Pour tester la présence d'un article dans la base de données, il suffit de tester si le lien associé à l'article est une clé de notre base de données.
  lien : Lien dont la présence dans la base de donnée sera testée
  data : Base de données
  """
  return lien in data.keys()

def liens_to_keep(url_rss, data):
  """
  Cette fonction retourne la liste des liens des articles d'un flux urss qui ne sont pas dans la base de données.
  On utilise pour cela la fonction isinjson().
  url_rss : Lien du flux RSS
  data : base de données
  """
  doc = fp.parse(url_rss)
  print(doc)
  return [dictio["link"] for dictio in doc["entries"] if not isinjson(dictio["link"], data)]

# Utiliser la fonction liens_to_keep pour scanner tous les articles de tous les flux RSS
liens_articles = []
for url in urls.values() :
  liens_articles += liens_to_keep(url, data_francetv)

print(len(liens_articles))

{'bozo': False, 'entries': [{'title': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration"}, 'summary': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants.", 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants."}, 'lastpublicationdate': 'Mon, 29 Jan 2024 21:10:47 +0100', 'class': 'Cms-Article', 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www

In [ ]:
data_francetv.keys()

dict_keys(['https://www.francetvinfo.fr/sante/prevention/eure-quand-les-elus-apprennent-a-se-defendre_3791635.html#xtor=RSS-3-[france]', 'https://www.francetvinfo.fr/france/video-sur-le-dos-d-un-aigle-pecheur-au-dessus-du-massif-du-mont-blanc-dont-les-glaciers-fondent-aujourd-hui-a-vue-d-oeil_3791583.html#xtor=RSS-3-[france]', 'https://www.francetvinfo.fr/societe/video-massif-du-mont-blanc-la-hausse-des-temperatures-fait-s-ecrouler-la-montagne-par-pans-entier-a-haute-altitude_3791537.html#xtor=RSS-3-[france]', 'https://www.francetvinfo.fr/replay-magazine/france-2/13h15/13h15-du-dimanche-19-janvier-2020_3766797.html#xtor=RSS-3-[france]', 'https://www.francetvinfo.fr/societe/loi-de-bioethique/loi-de-bioethique-les-opposants-a-la-pma-manifestent-a-paris_3791629.html#xtor=RSS-3-[france]', 'https://www.francetvinfo.fr/societe/loi-de-bioethique/loi-de-bioethique-les-anti-pma-a-nouveau-dans-la-rue_3791531.html#xtor=RSS-3-[france]', 'https://www.francetvinfo.fr/france/tendance-louer-ses-meuble

In [ ]:
liens_to_keep("https://www.francetvinfo.fr/france.rss", data_francetv)

{'bozo': False, 'entries': [{'title': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration"}, 'summary': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants.", 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants."}, 'lastpublicationdate': 'Mon, 29 Jan 2024 21:10:47 +0100', 'class': 'Cms-Article', 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www

['https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[france]',
 'https://www.francetvinfo.fr/france.rss#xtor=RSS-3-[fr

#### Question 2

In [ ]:
# Nous téléchargeons et regardons le code html du premier article de la base de donnée
import newspaper as np
article = np.Article(list(data_francetv.keys())[0])
article.download()
article.parse()
print(article.html)






<!DOCTYPE html>
<html lang="fr" class="is-mobile no-js">
<head>
        <title>Violences contre les maires : dans l&#039;Eure, des &eacute;lus apprennent &agrave; se d&eacute;fendre</title>
    <meta charset="UTF-8">
    <meta name="language" content="fr-FR">
    <meta name="viewport" content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0, maximum-scale=5.0">
    <meta name="description" content="&Agrave; deux mois des &eacute;lections municipales, l&#039;Association des maires ruraux de l&#039;Eure propose des cours d&#039;autod&eacute;fense aux &eacute;lus et aux secr&eacute;taires de mairie. Cette initiative est une premi&egrave;re en France." />
    <meta name="robots" content="index, follow, noarchive" />
    <meta name="googlebot" content="max-snippet:300, max-image-preview:large, max-video-preview:3">
    <meta name="google-play-app" content="app-id=fr.francetv.apps.info">
    <meta name="apple-itunes-app" content="app-id=348684697" />

      

Après avoir affiché le code HTML associé à l'article, nous constatons, qu'il est laborieux de naviguer dans ce code afin de trouver les éléments spécifiques tels que le titre de l'article ou le texte de l'article.

In [ ]:
# Afficher le texte de l'article grâce à la librairie newspaper
print(article.text)

À deux mois des élections municipales, l'Association des maires ruraux de l'Eure propose des cours d'autodéfense aux élus et aux secrétaires de mairie. Cette initiative est une première en France.

Beaucoup de maires et de secrétaires de mairie ont rencontré au cours de leurs mandats des problèmes de violence, essentiellement verbale. C'est le cas de Claire. Un jour, un homme est entré dans son bureau et a frappé très fort sur son bureau pour l'intimider. "J'ai réagi à l'instinct, j'ai essayé de discuter, mais ça s'est envenimé. Heureusement qu'une dame est arrivée à la permanence et a menacé d'appeler la gendarmerie", raconte Claire Carrère-Godebout, maire (SE) de Graveron-Sémerville (Eure).

Des professeurs de karaté et un psychologue

La plupart de ces élus ne savent pas comment se comporter face à une situation conflictuelle. Alors, l'Association des maires ruraux de l'Eure les initie à l'autodéfense, avec l'aide de professeurs de karaté. Et un psychologue leur a expliqué comment d

Nous pouvons récupérer le texte de l'article avec la méthode .text.
Nous constatons que la librairie newspaper est utile car elle nous permet d'obtenir certains éléments spécifiques désirés sans avoir à parcourir le code html.

#### Question 3

In [ ]:
def scrapping_rss(urls, data):
  """
  Permet de mettre à jour la base de données data avec les articles contenus dans chaque flux RSS de la liste urls
  urls: Liste de liens de flux RSS
  data: base de données
  """
  datacopy = data.copy()
  for category, url in urls.items() :
    print(category, url)
    liens_articles = liens_to_keep(url, data)
    print(f"Il y'a {len(liens_articles)} nouveaux articles")
    for lien in liens_articles :
      temp_dict = {}
      article = np.Article(lien)
      article.download()
      article.parse()
      temp_dict['title'] = article.title
      temp_dict['date'] = article.publish_date
      temp_dict['author'] = article.authors
      temp_dict['category'] = category
      temp_dict['content'] = article.text
      temp_dict['image_link'] = article.top_image

      datacopy[lien] = temp_dict

  return datacopy

In [ ]:
new_data_francetv = scrapping_rss(urls, data_francetv)

france https://www.francetvinfo.fr/france.rss
{'bozo': False, 'entries': [{'title': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration", 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Mayotte : des barrages routiers installés par des manifestants pour protester contre l'insécurité et l'immigration"}, 'summary': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants.", 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://www.francetvinfo.fr/france.rss', 'value': "Sur demande du préfet, les forces de l'ordre avaient entamé samedi le démantèlement de ces barrages. Mais cette opération a suscité la colère des manifestants."}, 'lastpublicationdate': 'Mon, 29 Jan 2024 21:10:47 +0100', 'class': 'Cms-Article', 'links': [{'rel': 'alterna

In [ ]:
urls.items() # urls est un dictionnaire dont la clé est la catégorie et la valeur le lien du flux RSS

dict_items([('france', 'https://www.francetvinfo.fr/france.rss'), ('europe', 'https://www.francetvinfo.fr/monde/europe.rss'), ('economie', 'https://www.francetvinfo.fr/economie/entreprises.rss')])

## Question 4

Pour faire une copie locale des images, on peut télécharger l'image à partir du lien et l'enregistrer localement. Puis nous devons stocker le chemin vers ce fichier dans la base de données.

## Question 5

Nous pourrions collecter d'autres informations sur un article, tel que la langue dans laquelle l'article est écrit, car un même article peut être traduit en plusieurs langue.

Nous pourrions également collecter le résumé de l'article. Pour cela, il faudrait que le résumé soit déjà rédiger par l'auteur et on aurait simplement le scrapper; sinon il faudrait utiliser un outil NLP qui permettrait de résumer le texte.

On pourrait aussi récupérer des articles de différents journaux nationaux ou régionaux et identifier le nombre d'articles qui parlent du même thème qu'un article de notre base de données. Un tel algorithme nécessiterait de faire des traitements NLP pour identifier le thème précis de chaque article récupéré. Pour répondre à cette problématique on pourrait utiliser une modélisation par thème (topic modeling) via une allocation latente de Dirichlet. On pourrait également vouloir comparer les thèmes au sein des différents journaux régionaux : les préoccupations sont-elles les mêmes sur tous les territoires ? sont-elles les mêmes au niveau national ?

Les librairies les plus adaptées pour ces traitements seraient feedparser et newspaper pour l'analyse du texte et/ou des journaux et spacy pour les tâches NLP.

## Question 7

Les étiquettes IOB (Inside, Outside, beginning) sont utilisées en NLP pour l'annotation des entités nommées. Elles permettent d'identifier un mot d'un texte comme faisant partie ou non d'un groupe de mots décrivant une entité nommée, en fonction de la place du mot considéré dans le groupe de mot. les trois types d'étiquettes sont :
- Inside : le mot fait partie d'une entité nommée mais n'est pas le premier mot du groupe de mots décrivant l'entité nommée
- Outside : le mot ne fait pas partie d'une entité nommée
- Beginning : le mot fait partie d'une entité nommée et est le premier mot du groupe de mots décrivant l'entité nommée.

Par exemple, pour la phrase "Barack Obama a été élu président", les annotations IOB pour la détection d'entités nommées pourraient être :

"Barack" : B-Personne

"Obama" : I-Personne

"a" : O

"été" : O

"élu" : O

"président" : O


## Question 8

In [ ]:
def entities_treatment(data):
  """
    Traite une collection d'articles pour extraire les entités nommées.

    Args:
    data (dict): Un dictionnaire (ou un objet similaire) où chaque clé correspond à un identifiant
                 d'article et chaque valeur est un dictionnaire contenant au moins une clé 'content'
                 avec le texte de l'article.

    Returns:
    list of list of dict: Une liste où chaque élément représente un article. Chaque article est
                          représenté par une liste d'entités. Chaque entité est un dictionnaire
                          contenant les clés 'text' (le texte de l'entité), 'start' (la position de
                          début de l'entité dans le texte de l'article), 'end' (la position de fin),
                          et 'label' (le type de l'entité).

    La fonction utilise spaCy pour traiter le texte de chaque article. Elle utilise un pipeline
    optimisé pour extraire uniquement les entités nommées, en désactivant les autres composants
    du pipeline spaCy tels que le tagger et le parser pour améliorer les performances. Chaque
    entité nommée est ensuite extraite et stockée dans une structure de données appropriée.

  """
  all_articles = [article["content"] for article in data.values()]
  pipeline = nlp.pipe(all_articles, disable=["tagger", "parser"])
  all_docs = [doc for doc in pipeline]
  article_entities = []
  for doc in all_docs:
    process = nlp(doc)
    entities = []
    for ent in process.ents:
        entities.append({"text": ent.text, "start": ent.start_char, "end": ent.end_char, "label": ent.label_})
    article_entities.append(entities)
  return article_entities

In [ ]:
article_entities = entities_treatment(new_data_francetv)

In [ ]:
len(article_entities)

2100

In [ ]:
article_entities

[[{'text': 'Claire', 'start': 155, 'end': 161, 'label': 'PER'},
  {'text': 'Claire Carrère-Godebout',
   'start': 437,
   'end': 460,
   'label': 'PER'},
  {'text': 'SE', 'start': 469, 'end': 471, 'label': 'LOC'},
  {'text': 'Graveron-Sémerville', 'start': 476, 'end': 495, 'label': 'LOC'},
  {'text': 'Eure', 'start': 497, 'end': 501, 'label': 'LOC'},
  {'text': "Association des maires ruraux de l'Eure",
   'start': 653,
   'end': 692,
   'label': 'ORG'},
  {'text': 'Christophe Micaux', 'start': 964, 'end': 981, 'label': 'PER'}],
 [{'text': 'Cliquez', 'start': 121, 'end': 128, 'label': 'LOC'},
  {'text': 'Victor', 'start': 204, 'end': 210, 'label': 'MISC'},
  {'text': 'Pygargue', 'start': 215, 'end': 223, 'label': 'MISC'},
  {'text': 'Haliaeetus', 'start': 241, 'end': 251, 'label': 'MISC'},
  {'text': 'massif du Mont-Blanc', 'start': 277, 'end': 297, 'label': 'LOC'},
  {'text': 'Europe', 'start': 321, 'end': 327, 'label': 'LOC'},
  {'text': 'Alpes', 'start': 841, 'end': 846, 'label': 'L

## Question 9

In [ ]:
from numpy import unique

all_entities = []
for liste in article_entities:
  all_entities+= liste

all_types = unique([dictio['label'] for dictio in all_entities])

def count_occur(typ):
  """
     Compte le nombre d'occurrences de chaque entité d'un type spécifique dans une collection d'entités.

    Args:
    typ (str): Le type d'entité pour lequel compter les occurrences.
               Par exemple, 'PER' pour les personnes, 'LOC' pour les lieux, etc.

    Returns:
    dict: Un dictionnaire où chaque clé est le texte d'une entité et chaque valeur est le nombre
          d'occurrences de cette entité dans la collection d'entités.

    La fonction parcourt toutes les entités dans une collection globale `all_entities`. Pour chaque
    entité, si son type correspond au type spécifié, la fonction incrémente un compteur associé à
    ce texte d'entité. Ce compteur est conservé dans un dictionnaire, permettant de suivre le nombre
    d'occurrences de chaque entité du type spécifié.
  """
  count_per_ent = {}
  for dictio in all_entities:
    if dictio['label']==typ:
      if dictio['text'] not in count_per_ent.keys():
        count_per_ent[dictio['text']]=1
      else:
        count_per_ent[dictio['text']]+=1
  return count_per_ent


In [ ]:
for typ in all_types:
  print(f"Type: {typ} \n{count_occur(typ)}")


Type: LOC 
{'SE': 1, 'Graveron-Sémerville': 1, 'Eure': 3, 'Cliquez': 39, 'massif du Mont-Blanc': 3, 'Europe': 354, 'Alpes': 10, 'Antarctique': 4, 'Amazonie': 2, 'glacier des Bossons': 1, 'Tour Ronde': 1, 'Chamonix': 1, 'chaîne des Alpes': 1, 'le Mont-Blanc': 1, 'département de la Haute-Savoie': 1, 'France': 795, "vallée d'Aoste": 1, 'Italie': 252, 'lesbiennes': 2, 'Paris': 371, 'la France': 396, "place de l'Opéra": 2, 'place de la Résistance': 1, '7e arrondissement de Paris': 1, 'Caroline': 6, 'Lyon': 48, 'Rhône': 22, 'Clermont-Ferrand': 6, 'Puy-de-Dôme': 7, 'lycée Blaise-Pascal': 1, 'Goldmen': 1, 'salle di Silo': 1, 'Marseille': 52, 'Français': 360, 'gare de Lyon': 2, 'Élysée': 18, 'État': 293, 'Lezat-sur-Leze': 1, 'Ariège': 3, 'Ariègeoise': 1, 'Appolonie': 2, 'Égypte\n\nAu village': 1, 'Égypte': 3, 'Zutkerque': 2, 'Pas-de-Calais': 29, 'CP': 1, 'Bron': 1, 'Oraison': 1, 'Alpes-de-Haute-Provence': 3, 'La France': 101, 'Hexagone': 40, 'rue de Trévise': 2, 'Saint-Brieuc': 1, 'IXème arrond

In [ ]:
count_LOC = count_occur("LOC")
count_MISC = count_occur("MISC")
count_ORG = count_occur("ORG")
count_PER = count_occur("PER")

In [ ]:
def sorted_dict(dictio):
  sorted_dictio = dict(sorted(dictio.items(), key=lambda item: item[1], reverse=True))
  return dict(list(sorted_dictio.items())[:20])

sorted_LOC = sorted_dict(count_LOC)
sorted_MISC = sorted_dict(count_MISC)
sorted_ORG = sorted_dict(count_ORG)
sorted_PER = sorted_dict(count_PER)

In [ ]:
top20_LOC = dict(list(sorted_LOC.items())[:20])
top20_LOC

{'France': 795,
 'Allemagne': 565,
 'la France': 396,
 'Paris': 371,
 'Français': 360,
 'Europe': 354,
 'Royaume-Uni': 298,
 'État': 293,
 'Italie': 252,
 'Etat': 232,
 'Chine': 207,
 'Grèce': 200,
 'Turquie': 167,
 "l'Europe": 163,
 'Bruxelles': 154,
 'Londres': 144,
 'Berlin': 134,
 'Espagne': 131,
 'Européens': 116,
 'Franceinfo': 112}

In [ ]:
top20_MISC = dict(list(sorted_MISC.items())[:20])
top20_MISC

{'Ça': 159,
 'Covid-19': 137,
 'Coronavirus': 123,
 'Premier ministre': 115,
 'Twitter': 100,
 'Brexit': 87,
 'Facebook': 76,
 'iOS & Android': 64,
 'Français': 49,
 'RADIOFRANCE': 32,
 'TPE': 28,
 'Pâques': 27,
 'Seconde Guerre mondiale': 23,
 '13 Heures': 22,
 'Grecs': 22,
 'TGV': 20,
 'ça': 18,
 'Bundesliga': 18,
 'FFP2': 17,
 'internet': 16}

In [ ]:
top20_ORG = dict(list(sorted_ORG.items())[:20])
top20_ORG

{'JT': 2081,
 'franceinfo': 476,
 'Covid-19': 454,
 'Union européenne': 318,
 'UE': 215,
 'AFP': 150,
 'Brexit': 134,
 'France Télévisions': 126,
 'Commission européenne': 104,
 'Renault': 96,
 'Travail': 88,
 'MAXPPP': 77,
 'CGT': 74,
 'Magazines': 67,
 'ANCV': 65,
 'ministre de la Santé': 61,
 'Alstom': 61,
 'Amazon': 54,
 'Finances': 50,
 'Airbus': 48}

In [ ]:
top20_PER = dict(list(sorted_PER.items())[:20])
top20_PER

{'Emmanuel Macron': 251,
 'Covid-19': 182,
 'Bruno Le Maire': 135,
 'Angela Merkel': 118,
 'Boris Johnson': 69,
 'Muriel Pénicaud': 59,
 'Laurent Desbonnets': 50,
 'président de la République': 47,
 'Thierry Breton': 44,
 'Ursula von der Leyen': 41,
 'Edouard Philippe': 40,
 'Donald Trump': 37,
 'Birkenau': 37,
 'Olivier Véran': 33,
 'Recep Tayyip Erdogan': 32,
 'Kyriakos Mitsotakis': 29,
 'Covid': 27,
 'George Floyd': 27,
 'Valéry Lerouge': 26,
 'Michel Barnier': 25}

## Question 10

In [ ]:
import pandas as pd

def co_occur_count(docs, ent1, ent2, typ1, typ2):
    """
    Compte le nombre de co-occurrences de deux entités spécifiques avec leurs types respectifs
    dans une collection de documents.

    Args:
    docs (list of list of dict): La collection de documents où chaque document est représenté
                                 par une liste d'entités. Chaque entité est un dictionnaire
                                 contenant les clés 'text' (le texte de l'entité) et 'label'
                                 (le type de l'entité).
    ent1 (str): Le texte de la première entité à rechercher.
    ent2 (str): Le texte de la seconde entité à rechercher.
    typ1 (str): Le type de la première entité (par exemple, 'PER' pour une personne).
    typ2 (str): Le type de la seconde entité (par exemple, 'LOC' pour un lieu).

    Returns:
    int: Le nombre de documents dans lesquels les deux entités spécifiées co-occurrent.

    """

    co_occur = 0
    for doc in docs:
        ent1_in_doc = any(entity['text'] == ent1 and entity['label'] == typ1 for entity in doc)
        ent2_in_doc = any(entity['text'] == ent2 and entity['label'] == typ2 for entity in doc)
        if ent1_in_doc and ent2_in_doc:
            co_occur += 1
    return co_occur



In [ ]:
co_occur_count(docs= article_entities, ent1 = "USA", ent2 = "LOC", typ1= "Joe Biden", typ2= "PER")

0

## Question 11

In [ ]:
words_type = []

# add persons
for per in top20_PER.keys():
  words_type.append({
      "word":per,
      "type":"PER"
  })

# add localisations
for loc in top20_LOC.keys():
  words_type.append({
      "word":loc,
      "type":"LOC"
  })

# add organisation
for org in top20_ORG.keys():
  words_type.append({
      "word":org,
      "type":"ORG"
  })

# add organisation
for misc in top20_MISC.keys():
  words_type.append({
      "word":misc,
      "type":"ORG"
  })

columns = ["word", "type"]
top_words = pd.DataFrame(words_type)
top_words.head()

,word,type
0,Emmanuel Macron,PER
1,Covid-19,PER
2,Bruno Le Maire,PER
3,Angela Merkel,PER
4,Boris Johnson,PER


In [ ]:
def get_pair_cooccurrence():
    # list of diffent pairs (ie different types)
    """
       Identifie et compte les co-occurrences de paires d'entités de types différents.

    Cette fonction ne prend aucun argument d'entrée direct, mais elle utilise 'top_words',
    un DataFrame Pandas global qui liste les entités et leurs types, ainsi que
    'article_entities', une liste globale de listes d'entités par document.

    Returns:
    list of dict: Une liste de dictionnaires, où chaque dictionnaire représente une paire d'entités.
                  Chaque dictionnaire contient les clés 'word1', 'word2' pour les textes des entités,
                  et 'count' pour le nombre de co-occurrences de cette paire dans l'ensemble des documents.

    La fonction parcourt chaque entité dans 'top_words', identifie d'autres entités de types différents
    et calcule le nombre de co-occurrences de chaque paire unique d'entités dans l'ensemble des
    documents 'article_entities'. Elle évite les doublons de paires et renvoie une liste de paires
    avec leurs comptes de co-occurrences

    """
    pairs = []
    for index, word in top_words.iterrows():
        # select the current word
        current_word = top_words.loc[index, "word"]
        current_type = top_words.loc[index, "type"]

        # select in the dataset the words that have a different type
        other_words = top_words[top_words["type"] != current_type]

        # iterate over all words of diffent type
        for second_index, second_word in other_words.iterrows():
            other_word = other_words.loc[second_index, "word"]
            other_type = other_words.loc[second_index, "type"]

            # check if the pair is not already present in the list of pairs
            present = False
            for pair in pairs:
                if pair['word1']==other_word and pair['word2']==current_word:
                    present = True
                    break

            if not present:
                pairs.append({
                    "word1": current_word,
                    "word2": other_word,
                    "count": co_occur_count(docs= article_entities, ent1 = current_word, ent2 = other_word, typ1= current_type, typ2= other_type)
                })
    return pairs

In [ ]:
pairs = get_pair_cooccurrence()
pairs.sort(key=lambda element: element['count'], reverse=True)
pairs[0:20]


[{'word1': 'France', 'word2': 'JT', 'count': 204},
 {'word1': 'Allemagne', 'word2': 'JT', 'count': 160},
 {'word1': 'Paris', 'word2': 'JT', 'count': 109},
 {'word1': 'la France', 'word2': 'JT', 'count': 107},
 {'word1': 'Covid-19', 'word2': 'France', 'count': 107},
 {'word1': 'Covid-19', 'word2': 'France', 'count': 107},
 {'word1': 'Français', 'word2': 'JT', 'count': 97},
 {'word1': 'France', 'word2': 'franceinfo', 'count': 96},
 {'word1': 'État', 'word2': 'JT', 'count': 80},
 {'word1': 'Italie', 'word2': 'JT', 'count': 74},
 {'word1': 'Franceinfo', 'word2': 'France Télévisions', 'count': 69},
 {'word1': 'Royaume-Uni', 'word2': 'Union européenne', 'count': 68},
 {'word1': 'Europe', 'word2': 'Union européenne', 'count': 67},
 {'word1': 'Franceinfo', 'word2': 'Magazines', 'count': 67},
 {'word1': 'Europe', 'word2': 'JT', 'count': 66},
 {'word1': 'Chine', 'word2': 'JT', 'count': 65},
 {'word1': 'Covid-19', 'word2': 'JT', 'count': 64},
 {'word1': 'Angela Merkel', 'word2': 'Allemagne', 'cou

Les entités comme "France", "JT", "Allemagne" apparaissent fréquemment, indiquant une forte présence dans les documents analysés.

Certaines paires, comme "Royaume-Uni" et "Union européenne", peuvent indiquer des discussions fréquentes sur des sujets politiques ou géographiques spécifiques.

Des entrées en double, comme 'Covid-19' et 'France', suggèrent qu'il pourrait y avoir des améliorations à apporter dans la logique de vérification des doublons dans la fonction.

#### Question 12

In [ ]:
all_articles = [article["content"] for article in new_data_francetv.values()]
articles = [nlp(article) for article in all_articles]


In [ ]:
all_sents = []
for article in articles:
  all_sents += list(article.sents)
all_sents

[Beaucoup de maires et de secrétaires de mairie ont rencontré au cours de leurs mandats des problèmes de violence, essentiellement verbale.,
 C'est le cas de Claire.,
 Un jour, un homme est entré dans son bureau et a frappé très fort sur son bureau pour l'intimider.,
 "J'ai réagi à l'instinct, j'ai essayé de discuter, mais ça s'est envenimé.,
 Heureusement qu'une dame est arrivée à la permanence et a menacé d'appeler la gendarmerie", raconte Claire Carrère-Godebout, maire (SE) de Graveron-Sémerville (Eure).
 
 Des professeurs de karaté et un psychologue
 
 La plupart de ces élus ne savent pas comment se comporter face à une situation conflictuelle.,
 Alors, l'Association des maires ruraux de l'Eure les initie à l'autodéfense, avec l'aide de professeurs de karaté.,
 Et un psychologue leur a expliqué comment désamorcer un différent.,
 "C'est nécessaire qu'ils aient confiance en eux donc, à partir de là, je pense que cet atelier ne peut avoir que des bénéfices", explique Christophe Micaux

In [ ]:
import pandas as pd

def co_occur_count_sent(all_sents, ent1, ent2, typ1, typ2):
    """
    Compte le nombre de co-occurrences au sein d'une même phrase de deux entités spécifiques avec leurs types respectifs
    dans une collection de documents.

    Args:
    all_sents (list of list of dict): La collection de phrase à considérer
    ent1 (str): Le texte de la première entité à rechercher.
    ent2 (str): Le texte de la seconde entité à rechercher.
    typ1 (str): Le type de la première entité (par exemple, 'PER' pour une personne).
    typ2 (str): Le type de la seconde entité (par exemple, 'LOC' pour un lieu).

    Returns:
    int: Le nombre de documents dans lesquels les deux entités spécifiées co-occurrent.

    """
    co_occurrence_count = 0
    for doc in all_sents:
      if ent1 in doc.text and ent2 in doc.text:
          co_occurrence_count += 1
    return co_occurrence_count



In [ ]:
co_occur_count_sent(all_sents, ent1 = "président", ent2 = "LOC", typ1= "Emmanuel Macron", typ2= "PER")

0



# Question 13

In [ ]:
nlp.add_pipe('sentencizer')

In [ ]:
from spacy.pipeline import sentencizer

def create_documents():
    '''
    Same as previous create_documents() method but here we study the sentences of each document.
    Here we iterate over a set of document. For each document we use the sentencizer pipeline.
    It enables us to cut a text as a set of sentences. For each sentence, we look for the entities.
    Then if there is one or several verbs in between the first and last entities we keep them in a
    list in order to define the relation in between them.

    Returns
    -------
    list
        List of documents
    '''
    content = nlp.pipe(all_articles, disable=["tagger", "parser"])
    documents = []
    i = 0
    # creating a list of dictionnaries where each item represents a document
    for document in content:
        # get all entities separated by one verb
        list_sentences = []
        for sentence in document.sents:
            # list of entities
            list_entity = []
            # get all entities
            entity = [entity.text for entity in sentence.ents]
            # if there is at least 2 entities, we look for the verbs
            if len(entity) > 1:
                # list of all first positions
                first_position = [entity.start for entity in sentence.ents]
                # start of first entity
                first = first_position[0]
                # list of all last positions
                last_position = [entity.end for entity in sentence.ents]
                # end position of last entity
                last = last_position[-1]

                # list of verbs in between two entities
                # get the verbs after the first entity position and before the last entity of the sentence
                verb = [token.lemma_ for token in sentence[first - sentence.start : last - sentence.start ] if token.pos_ == "VERB"]

                # make a list of all entitites in the sentence
                for entity in sentence.ents:
                    list_entity.append({
                        "text": entity.text,
                        "type": entity.label_
                        # "span": [entity.start, entity.end]
                        })
                # if there is at least one verb, append the information in the list
                if verb:
                    list_sentences.append({
                        "sentence": sentence.text,
                        "entity":list_entity,
                        "verb":verb,
                        "span": [sentence.start, sentence.end]
                    })
        # add all the information about the document in the list
        documents.append({
            "doc_id":i,
            "sentence":list_sentences
        })
        i+=1
    return documents

In [ ]:
documents =  create_documents()
documents[0]

{'doc_id': 0, 'sentence': []}

In [ ]:
def words_with_verb(word1:str, type1:str, word2:str, type2:str, documents=documents):
    '''
    This function looks for the verb relation in between two entitites.

    Parameters
    ----------
    word1 : str
        First entity
    type1 : str
        Type of the first entity
    word2 : str
        Second entity
    type2 : str
        Type of the second entity
    documents : list
        List of documents to iterate over

    Returns
    -------
    dict
        set of all verbs found in between the two entities
    '''
    # empty list of verbs
    verbs = []
    # iterate over all documents
    for doc in documents:
        # create list of all entities in the sentences
        sentence_entities = [doc["sentence"][i]["entity"] for i in range(len(doc['sentence']))]
        index_found = 0
        # iterate over all sentence entities
        for entity in sentence_entities:
            word1_found = False
            word2_found = False
            for word_index in range(len(entity)):
                # check if word1 found and type1 correct
                if word1 in entity[word_index]['text'] and type1.lower() == entity[word_index]['type'].lower():
                    word1_found = True
            for word_index in range(len(entity)):
                # check if word2 found and type2 correct
                if word2 in entity[word_index]['text'] and type2.lower() == entity[word_index]['type'].lower() :
                    word2_found = True
            index_found += 1
            # if both words found add to list of verbs
            if word1_found == True and word2_found == True:
                verbs.append(
                    doc["sentence"][index_found - 1]["verb"]
                )
    return set([val for sublist in verbs for val in sublist])

# Question 14

In [ ]:
print(words_with_verb(word1='France', type1='LOC', word2='Emmanuel Macron', type2="PER"))
print(words_with_verb(word1='Washington', type1='LOC', word2='Etats-Unis', type2="LOC"))
print(words_with_verb(word1='Europe', type1='LOC', word2='Afrique', type2="LOC"))

{'lancer', 'entendre', 'paraître', 'rappeler', 'devoir', 'endiguer', 'prendre', 'obtenir', 'éviter', 'jouer', 'réunir', 'parler', 'sévir', 'pouvoir', 'efforcer', 'souligner', 'vanter', 'agir', 'rapporter', 'annoncer', 'engager', 'provoquer', 'expliquer', 'avoir', 'recevoir', 'préside', 'rendre', 'partir', 'affirmer', 'adresse', 'aller', 'faire', 'dénoncer', 'lutter', 'commander', 'mener', 'exprimer', 'covid-19', 'demander', 'fermer', 'inciter', 'charger', 'téléviser', 'saluer', 'démontrer', 'prévenir', 'produire'}
set()
{'covid-19', 'posséder', 'avoir', 'accueillir', 'savoir', 'toucher', 'trouver'}


# Question 15



Les couples d'entités et les verbes associés identifiés peuvent être extrêmement utiles pour découvrir de nouvelles relations et grouper des relations de même nature dans un ensemble de documents.

En examinant les verbes associés à des couples d'entités, on peut identifier des relations spécifiques qui n'avaient peut-être pas été explicitement annotées dans les documents.
Cela peut aider à révéler des liens subtils entre les entités et à découvrir des informations non évidentes.

En regroupant les relations basées sur des verbes similaires, on peut identifier des motifs récurrents dans les documents.
Cela peut faciliter la création de catégories ou de thèmes généraux, ce qui peut être utile pour l'organisation et la compréhension des données.

En analysant les verbes associés à des entités, on peut identifier des événements ou des tendances spécifiques liés à ces entités.
Cela peut être utile pour la surveillance des actualités, la veille concurrentielle, etc.

# Question 16

L'utilisation de l'arbre de dépendance syntaxique peut affiner l'analyse des relations entre les entités en capturant la structure grammaticale des phrases. En effet, cela permettrait de trouver le verbe principal de la phrase et ainsi caractériser au mieux la relation entre deux entités. Les dépendances syntaxiques décrivent comment les mots d'une phrase sont connectés les uns aux autres par des relations grammaticales, telles que les relations sujet-verbe, objet-direct, etc.